<a href="https://colab.research.google.com/github/SeyoungYoo/DLforEveryone/blob/main/Season01/ML_Lab04_2_Loading_Data_from_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [104]:
!pip install tensorflow==1.15.0

##**파일에서 데이터 가져오기**

###**numpy 사용**

In [105]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(777) # for reproducibility

tf.set_random_seed(777)는 모든 기기에서 동일한 난수를 발생시키기 위함

In [106]:
xy = np.loadtxt('data-01-test-score.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

In [107]:
# Make sure the shape and data are OK
print(x_data.shape, x_data, len(x_data))
print(y_data.shape, y_data)

(6, 3) [[ 73.  80.  75.]
 [ 93.  88.  93.]
 [ 89.  91.  90.]
 [ 96.  98. 100.]
 [ 73.  66.  70.]
 [ 53.  46.  55.]] 6
(6, 1) [[152.]
 [185.]
 [180.]
 [196.]
 [142.]
 [101.]]


In [108]:
# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None,3])
Y = tf.placeholder(tf.float32, shape=[None,1])

In [109]:
W = tf.Variable(tf.random_normal([3,1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

In [110]:
# Hypothesis
hypothesis = tf.matmul(X, W) + b

In [111]:
# Simplified cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

In [112]:
# Minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

In [113]:
# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables in the grpah.
sess.run(tf.global_variables_initializer())
# Set up feed_dict variables inside the loop.
for step in range(2001):
  cost_val, hy_val, _ = sess.run(
      [cost, hypothesis, train],
      feed_dict={X: x_data, Y: y_data})
  if step % 10 == 0:
    print(step, "Cost: ", cost_val, "\nPrediction:\n", hy_val)

0 Cost:  69383.96 
Prediction:
 [[-100.601845]
 [-111.113365]
 [-114.66445 ]
 [-122.809456]
 [ -84.09649 ]
 [ -57.829273]]
10 Cost:  20.2633 
Prediction:
 [[144.95445]
 [184.0824 ]
 [176.16753]
 [193.9097 ]
 [141.06879]
 [108.15358]]
20 Cost:  16.992172 
Prediction:
 [[146.6351 ]
 [186.07327]
 [178.14452]
 [196.05861]
 [142.58336]
 [109.26029]]
30 Cost:  16.902998 
Prediction:
 [[146.664  ]
 [186.0784 ]
 [178.16518]
 [196.07703]
 [142.58328]
 [109.25042]]
40 Cost:  16.81454 
Prediction:
 [[146.6817 ]
 [186.07016]
 [178.1726 ]
 [196.08102]
 [142.573  ]
 [109.23307]]
50 Cost:  16.726667 
Prediction:
 [[146.69925]
 [186.06186]
 [178.17992]
 [196.0849 ]
 [142.5627 ]
 [109.21571]]
60 Cost:  16.63928 
Prediction:
 [[146.71675 ]
 [186.05356 ]
 [178.1872  ]
 [196.08878 ]
 [142.55241 ]
 [109.198395]]
70 Cost:  16.552538 
Prediction:
 [[146.73418 ]
 [186.04527 ]
 [178.19443 ]
 [196.09262 ]
 [142.54216 ]
 [109.181145]]
80 Cost:  16.466345 
Prediction:
 [[146.75156 ]
 [186.03705 ]
 [178.20168 ]
 [

In [114]:
# Ask my score
print("Your score will be ", sess.run(hypothesis, feed_dict={X:[[100,70,101]]}))
print("Other scores will be ", sess.run(hypothesis, feed_dict={X:[[60,70,110],[90,100,80]]}))

Your score will be  [[195.54945]]
Other scores will be  [[177.37123]
 [170.37453]]


###**batch 사용**

In [115]:
import tensorflow as tf
tf.set_random_seed(777)  # for reproducibility

In [116]:
filename_queue = tf.train.string_input_producer(
    ['data-01-test-score.csv'], shuffle=False, name='filename_queue')

In [117]:
# txt 파일 읽어오기
reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

In [118]:
# Default valeus, in case of empty columns. Also specifies the type of the decoded result.
# 데이터 형태 맞춰주기
record_defaults = [[0.], [0.], [0.], [0.]]
xy = tf.decode_csv(value, record_defaults=record_defaults)

In [119]:
# collect batches of csv in 
train_x_batch, train_y_batch = \
    tf.train.batch([xy[0:-1], xy[-1:]], batch_size=10)

In [120]:
# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 3])
Y = tf.placeholder(tf.float32, shape=[None, 1])

In [121]:
W = tf.Variable(tf.random_normal([3, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

In [122]:
# Hypothesis
hypothesis = tf.matmul(X, W) + b

In [123]:
# Simplified cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

In [124]:
# Minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

In [125]:
# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

In [126]:
# Start populating the filename queue.
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

In [127]:
for step in range(2001):
    x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
    cost_val, hy_val, _ = sess.run(
        [cost, hypothesis, train], feed_dict={X: x_batch, Y: y_batch})
    if step % 10 == 0:
        print(step, "Cost: ", cost_val, "\nPrediction:\n", hy_val)

0 Cost:  17326.836 
Prediction:
 [[31.878458]
 [42.126556]
 [39.376747]
 [43.513084]
 [32.812458]
 [26.112646]
 [31.878458]
 [42.126556]
 [39.376747]
 [43.513084]]
10 Cost:  6.483055 
Prediction:
 [[140.69208]
 [105.63453]
 [149.53651]
 [183.56047]
 [178.72455]
 [195.2633 ]
 [140.69208]
 [105.63453]
 [149.53651]
 [183.56047]]
20 Cost:  5.688437 
Prediction:
 [[179.57188 ]
 [196.18398 ]
 [141.34187 ]
 [106.108864]
 [150.25658 ]
 [184.41405 ]
 [179.57188 ]
 [196.18398 ]
 [141.34187 ]
 [106.108864]]
30 Cost:  3.4184308 
Prediction:
 [[150.17313]
 [184.3017 ]
 [179.46758]
 [196.06828]
 [141.25491]
 [106.04011]
 [150.17313]
 [184.3017 ]
 [179.46758]
 [196.06828]]
40 Cost:  5.973648 
Prediction:
 [[141.3684 ]
 [106.11928]
 [150.3079 ]
 [184.4521 ]
 [179.6219 ]
 [196.23434]
 [141.3684 ]
 [106.11928]
 [150.3079 ]
 [184.4521 ]]
50 Cost:  5.6416116 
Prediction:
 [[179.58748]
 [196.19475]
 [141.33546]
 [106.09033]
 [150.2835 ]
 [184.41064]
 [179.58748]
 [196.19475]
 [141.33546]
 [106.09033]]
60 C

In [128]:
coord.request_stop()
coord.join(threads)

In [129]:
# Ask my score
print("Your score will be ",
      sess.run(hypothesis, feed_dict={X: [[100, 70, 101]]}))

print("Other scores will be ",
      sess.run(hypothesis, feed_dict={X: [[60, 70, 110], [90, 100, 80]]}))

Your score will be  [[186.96579]]
Other scores will be  [[164.86417]
 [176.91783]]
